# PINNs

Consideremos la misma ecuación diferencial, resuelta en el notebook anterior con el método de residuos de ponderación: 

$$\frac{d^2u}{dx^2} + u + x = 0 \enspace , $$

con condiciones de frontera $u(0)=u(1)=0$.

In [1]:
# Import NumPy for numerical operations
import numpy as np

# Import Matplotlib for plotting
import matplotlib.pyplot as plt

# Import PyTorch for building and training neural networks
import torch
import torch.nn as nn
import torch.optim as optim

# Import the time module to time our training process
import time

# Import a utility module for additional plotting functions
import utils_plots

# Ignore Warning Messages
import warnings
warnings.filterwarnings("ignore")

La solución a este problema de valores de la frontera es:

$$u_e(x) = \frac{\sin(x)}{\sin(1)} - x \enspace .$$

In [2]:
x = torch.linspace(0,1,100).view(-1,1)
x_physics = torch.linspace(0,1,100).view(-1,1).requires_grad_(True)
u_e = torch.sin(x)/torch.sin(torch.tensor(1)) - x

# Create a figure for the plot
fig = plt.figure(figsize=(3.5, 2.0))

plt.plot(x,u_e,color='gray', linewidth=2)

# Set the title and labels for the x and y axes
plt.xlabel('$x$')
plt.ylabel('$u_{e}$')

# Display the plot
plt.show()

En el siguiente bloque de código, definimos nuestro modelo de red neuronal. Este modelo es una red neuronal de alimentación directa simple con dos capas ocultas, cada una conteniendo **50 neuronas**. La función de activación utilizada es la tangente hiperbólica (`tanh`). También instanciamos el modelo y definimos el optimizador (**Adam**) y la función de pérdida (**Error Cuadrático Medio**) que se utilizarán durante el entrenamiento.

In [7]:
# Define a neural network class with three fully connected layers
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(1, 10)
        self.layer2 = nn.Linear(10, 10)
        self.output_layer = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.tanh(self.layer1(x))
        x = torch.tanh(self.layer2(x))
        x = self.output_layer(x)
        return x
    
# Create an instance of the neural network
model = NeuralNetwork()

# Define an optimizer (Adam) for training the network
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define a loss function (Mean Squared Error) for training the network
loss_func = nn.MSELoss()

En el siguiente bloque de código, entrenamos nuestra red neuronal. Ejecutamos el proceso de entrenamiento durante **2000 iteraciones**.

In [8]:
torch.manual_seed(123)

files = []
losses = []
for i in range(2_001):
    optimizer.zero_grad()
    uh = model(x_physics)
    
    # Calculate physics loss
    dudx  = torch.autograd.grad(uh, x_physics, torch.ones_like(uh), create_graph=True)[0]  # computes du/dx
    dudx2 = torch.autograd.grad(dudx, x_physics, torch.ones_like(dudx), create_graph=True)[0]  # computes d^2u/dx^2
    physics = dudx2 + uh + x_physics 
    loss_physics = loss_func(physics,torch.tensor([0.]))
    
    # Calculate boundary conditions loss
    loss_bc = loss_func(uh[0], torch.tensor([0.])) + loss_func(uh[-1], torch.tensor([0.]))
    
    # Calculate loss as the sum of phyiscs and boundary losses
    loss = loss_physics + loss_bc    
    
    # backpropagate loss
    loss.backward()
    optimizer.step()
    losses.append(loss)

    # Print progress every 1000 iterations
    if i % 100 == 0:
        print(f"Iteration {i}, Loss: {loss.item()}")

Iteration 0, Loss: 0.3766207695007324
Iteration 100, Loss: 0.1318909227848053
Iteration 200, Loss: 0.022006485611200333
Iteration 300, Loss: 0.005070406477898359
Iteration 400, Loss: 0.0020514661446213722
Iteration 500, Loss: 0.0012448420748114586
Iteration 600, Loss: 0.0008179526776075363
Iteration 700, Loss: 0.0005392422317527235
Iteration 800, Loss: 0.0003599501505959779
Iteration 900, Loss: 0.00024847930762916803
Iteration 1000, Loss: 0.00018106677453033626
Iteration 1100, Loss: 0.00014112243661656976
Iteration 1200, Loss: 0.00011749359691748396
Iteration 1300, Loss: 0.0001030211424222216
Iteration 1400, Loss: 9.340166434412822e-05
Iteration 1500, Loss: 8.624117617728189e-05
Iteration 1600, Loss: 8.031049219425768e-05
Iteration 1700, Loss: 7.502570952055976e-05
Iteration 1800, Loss: 7.013002323219553e-05
Iteration 1900, Loss: 6.551727710757405e-05
Iteration 2000, Loss: 6.114653660915792e-05


In [9]:
# Convert each tensor to a Python number
number_list = [t.item() for t in losses]

# Convert the list of numbers to a numpy array
losses = np.array(number_list)
# Create a figure for the plot
fig = plt.figure(figsize=(3.5, 2.5))

# Plot the loss values recorded during training
plt.semilogy(losses, color='gray', linewidth=2)

# Set the labels for the x and y axes
plt.xlabel('Iteration')
plt.ylabel('Log Loss')

# Set the title for the plot
plt.title('Training Progress (Log Scale)', fontsize=10)

# Display the grid
plt.grid(True)

# Display the plot
plt.show()

In [10]:
# Generate predictions
predictions = model(x_physics)

# Convert tensors to numpy arrays for plotting
x = x_physics.detach().numpy()
u_pred = predictions.detach().numpy()

# Create a figure for the plot
fig = plt.figure(figsize=(3.5, 2.0))

# Plot the predictions
plt.plot(x, u_pred, color='gray', linewidth=2, label='Prediction')
plt.plot(x, u_e, color='blue', linewidth=2, linestyle='dashed', label='Exact')

# Set the title and labels
plt.xlabel('$x$')
plt.ylabel('$u$')

# Add a legend
plt.legend()

# Increase the size of the plot for better visibility
plt.rcParams["figure.figsize"] = [10, 6]

# Display the plot
plt.show()